In [13]:
# Primero vemos algunas filas del texto preprocesado
import pandas as pd

df = pd.read_parquet("../data/train_preprocessed.parquet")
df.head(10)[["messages", "text_clean", "tokens", "lemmas"]]

,messages,text_clean,tokens,lemmas
0,Germany!\n\nJust the person I want to speak wi...,germany just the person i want to speak with i...,"['germany', 'person', 'want', 'speak', 'somewh...","['germany', 'person', 'want', 'speak', 'somewh..."
1,"You've whet my appetite, Italy. What's the sug...",youve whet my appetite italy whats the suggestion,"['ve', 'whet', 'appetite', 'italy', 's', 'sugg...","['ve', 'whet', 'appetite', 'italy', 's', 'sugg..."
2,It seems like there are a lot of ways that cou...,it seems like there are a lot of ways that cou...,"['like', 'lot', 'ways', 'wrong', 'nt', 'france...","['like', 'lot', 'way', 'wrong', 'not', 'france..."
3,"Yeah, I can’t say I’ve tried it and it works, ...",yeah i can t say i ve tried it and it works ca...,"['yeah', 't', 've', 'tried', 'works', 'cause',...","['yeah', 't', 've', 'try', 'work', 'cause', 'v..."
4,I am just sensing that you don’t like this ide...,i am just sensing that you don t like this ide...,"['sensing', 'don', 't', 'like', 'idea', 'shall...","['sense', 'don', 't', 'like', 'idea', 'shall',..."
5,Any thoughts?,any thoughts,['thoughts'],['thought']
6,"Sorry Italy I've been away doing, um, German t...",sorry italy ive been away doing um german thin...,"['sorry', 'italy', 've', 'away', 'um', 'german...","['sorry', 'italy', 've', 'away', 'um', 'german..."
7,"I don't think I'm ready to go for that idea, h...",i dont think im ready to go for that idea howe...,"['nt', 'think', 'm', 'ready', 'idea', 'd', 'go...","['not', 'think', 'm', 'ready', 'idea', 'd', 'g..."
8,But I think I can get over “Lagergate” and we ...,but i think i can get over lagergate and we ca...,"['think', 'lagergate', 'friends', 'right', 'th...","['think', 'lagergate', 'friend', 'right', 'thi..."
9,We haven't even passed a season yet and you ha...,we havent even passed a season yet and you hav...,"['nt', 'passed', 'season', 'reliable', 'ally',...","['not', 'pass', 'season', 'reliable', 'ally', ..."


In [14]:
# Volvemos a visualizar algunas lineas del texto procesado despues de arreglar los errores(emojis y letras sueltas)
import pandas as pd

df = pd.read_parquet("../data/train_preprocessed.parquet")
df.head(10)[["messages", "text_clean", "tokens", "lemmas"]]

,messages,text_clean,tokens,lemmas
0,Germany!\n\nJust the person I want to speak wi...,germany just the person i want to speak with i...,"['germany', 'person', 'want', 'speak', 'somewh...","['germany', 'person', 'want', 'speak', 'somewh..."
1,"You've whet my appetite, Italy. What's the sug...",youve whet my appetite italy whats the suggestion,"['ve', 'whet', 'appetite', 'italy', 's', 'sugg...","['ve', 'whet', 'appetite', 'italy', 's', 'sugg..."
2,It seems like there are a lot of ways that cou...,it seems like there are a lot of ways that cou...,"['like', 'lot', 'ways', 'wrong', 'nt', 'france...","['like', 'lot', 'way', 'wrong', 'not', 'france..."
3,"Yeah, I can’t say I’ve tried it and it works, ...",yeah i can t say i ve tried it and it works ca...,"['yeah', 't', 've', 'tried', 'works', 'cause',...","['yeah', 't', 've', 'try', 'work', 'cause', 'v..."
4,I am just sensing that you don’t like this ide...,i am just sensing that you don t like this ide...,"['sensing', 'don', 't', 'like', 'idea', 'shall...","['sense', 'don', 't', 'like', 'idea', 'shall',..."
5,Any thoughts?,any thoughts,['thoughts'],['thought']
6,"Sorry Italy I've been away doing, um, German t...",sorry italy ive been away doing um german thin...,"['sorry', 'italy', 've', 'away', 'um', 'german...","['sorry', 'italy', 've', 'away', 'um', 'german..."
7,"I don't think I'm ready to go for that idea, h...",i dont think im ready to go for that idea howe...,"['nt', 'think', 'm', 'ready', 'idea', 'd', 'go...","['not', 'think', 'm', 'ready', 'idea', 'd', 'g..."
8,But I think I can get over “Lagergate” and we ...,but i think i can get over lagergate and we ca...,"['think', 'lagergate', 'friends', 'right', 'th...","['think', 'lagergate', 'friend', 'right', 'thi..."
9,We haven't even passed a season yet and you ha...,we havent even passed a season yet and you hav...,"['nt', 'passed', 'season', 'reliable', 'ally',...","['not', 'pass', 'season', 'reliable', 'ally', ..."


In [15]:
# Ahora vemos el tamaño del vocabulario de los modelos BoW y TF-IDF
import joblib
cv = joblib.load("../diplomacy/models/representations/bow_vectorizer.joblib")
tfidf = joblib.load("../diplomacy/models/representations/tfidf_vectorizer.joblib")

print("Vocabulario BoW:", len(cv.vocabulary_))
print("Vocabulario TF-IDF:", len(tfidf.vocabulary_))

# Mostrar algunas palabras del vocabulario
list(cv.vocabulary_.keys())[:20]

Vocabulario BoW: 8950
Vocabulario TF-IDF: 8950


['germany',
 'just',
 'the',
 'person',
 'want',
 'to',
 'speak',
 'with',
 'have',
 'somewhat',
 'crazy',
 'idea',
 'that',
 've',
 'always',
 'wanted',
 'try',
 'but',
 'never',
 'actually']

In [16]:
# Vamos a ver ejemplos de embeddings Word2Vec
from gensim.models import Word2Vec
w2v = Word2Vec.load("../diplomacy/models/embeddings/word2vec.model")

# Número de palabras aprendidas
print("Tamaño del vocabulario Word2Vec:", len(w2v.wv.key_to_index))

# Ver los términos más similares a una palabra
w2v.wv.most_similar("support", topn=5)

Tamaño del vocabulario Word2Vec: 2025


[('cut', 0.8592541217803955),
 ('supporting', 0.8437879085540771),
 ('supported', 0.839428722858429),
 ('supports', 0.8161010146141052),
 ('bump', 0.8006856441497803)]

In [17]:
# Ahora vemos ejemplos de embeddings FastText 
from gensim.models import FastText
ft = FastText.load("../diplomacy/models/embeddings/fasttext.model")
ft.wv.most_similar("attack", topn=5)

[('attacks', 0.9859697222709656),
 ('supply', 0.9812034368515015),
 ('supposedly', 0.9783168435096741),
 ('supposed', 0.9766971468925476),
 ('suppose', 0.9713596105575562)]

In [18]:
# Ahora vemos la forma de los embeddings BERT
import numpy as np

bert_train = np.load("../diplomacy/models/embeddings/bert_train.npy")
print("Shape BERT embeddings:", bert_train.shape)

Shape BERT embeddings: (11894, 768)


In [ ]:
# Medimos la cobertura de los modelos de embeddings
import pandas as pd
from gensim.models import Word2Vec
from collections import Counter
import numpy as np
import ast

# Función auxiliar para medir cobertura
def compute_coverage(tokens_list, model):
    total, covered = 0, 0
    oov = Counter()
    for tokens in tokens_list:
        for t in tokens:
            total += 1
            if t in model.wv:
                covered += 1
            else:
                oov[t] += 1
    coverage = covered / total * 100
    return coverage, oov

# Cargar dataset preprocesado
df = pd.read_parquet("../data/train_preprocessed.parquet")

# Asegurar que tokens sean listas (no strings)
if isinstance(df["tokens"].iloc[0], str):
    df["tokens"] = df["tokens"].apply(ast.literal_eval)

w2v = Word2Vec.load("../diplomacy/models/embeddings/word2vec.model")
new_coverage, new_oov = compute_coverage(df["tokens"], w2v)
print(f"Cobertura Word2Vec (después de limpieza): {new_coverage:.2f}%")

# Mostrar algunas palabras fuera del vocabulario
print("\nPalabras OOV más frecuentes:")
for word, count in new_oov.most_common(15):
    print(f"{word:<15} {count}")


Cobertura Word2Vec (después de limpieza): 92.13%

Palabras OOV más frecuentes:
approaching     4
hop             4
explicitly      4
weakening       4
informed        4
woke            4
bridge          4
explanation     4
sets            4
argument        4
willingness     4
history         4
confidence      4
encouraging     4
betraying       4


In [19]:
# Finalmente, vemos la distribución de la longitud de los textos en tokens
df["len_tokens"] = df["tokens"].apply(len)
df["len_tokens"].describe()

count    11894.000000
mean        75.943165
std         74.447143
min          2.000000
25%         28.000000
50%         54.000000
75%         99.000000
max       1098.000000
Name: len_tokens, dtype: float64

In [23]:
# Visualización de la distribución
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import scipy.sparse as sp

# Cargar TF-IDF
X = sp.load_npz("../diplomacy/models/representations/X_tfidf_train.npz")
y = pd.read_parquet("../data/train_preprocessed.parquet")["sender_labels"]

# Convertir etiquetas a numéricas
y = (y == "True").astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       104
           1       0.96      1.00      0.98      2275

    accuracy                           0.96      2379
   macro avg       0.48      0.50      0.49      2379
weighted avg       0.91      0.96      0.93      2379



c:\Users\Oihane\Desktop\NLP\Entrega2\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Oihane\Desktop\NLP\Entrega2\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Oihane\Desktop\NLP\Entrega2\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()}